In [1]:
%run "./problems.ipynb"
%run "./topo_physics.ipynb"
%run "./utils.ipynb"
import scipy.sparse
import scipy.sparse.linalg
from matplotlib import pyplot as plt
import xarray as xr
from skimage.util import view_as_windows

In [2]:
def generate_square_problem(dim, factor):
  '''
  在一个正方形设计域中，在外周的节点上生成随机的固定支座和力。(借鉴https://arxiv.org/abs/2106.13652 ，P10-11)
  为了保证维度缩减factor后这些边界条件节点仍能精确地对应，在随机选择节点时，候选节点只包含缩减后仍存在的节点。
  例如，如果要使结构可以缩减2、3、4倍，则factor应设置为最小公倍数12。
  '''
  num_candidate_points = dim//factor * 4
  ppl = dim//factor # points per length，即在每条边上有多少可供选择的点
  X, Y = 0, 1
 
  normals = np.zeros((dim + 1, dim + 1, 2))
  forces = np.zeros((dim + 1, dim + 1, 2))
  # 2-3 fixed boundaries, 2-3 unit forces in Y and X direction
  bc_points = np.random.choice(num_candidate_points, np.random.randint(2,4), replace=False) # 随机选择固支节点
  for point in bc_points:
    if point < ppl:
      normals[dim*point//ppl, 0, X] = 1
      normals[dim*point//ppl, 0, Y] = 1
    elif point < ppl*2:
      normals[dim, dim*(point%ppl)//ppl, X] = 1
      normals[dim, dim*(point%ppl)//ppl, Y] = 1
    elif point < ppl*3:
      normals[dim*(point%ppl)//ppl, dim, X] = 1
      normals[dim*(point%ppl)//ppl, dim, Y] = 1
    else:
      normals[0, dim*(point%ppl)//ppl, X] = 1
      normals[0, dim*(point%ppl)//ppl, Y] = 1
  
  f_candidates = np.asarray(list(set(np.arange(num_candidate_points)) - set(bc_points))) # 确保力的节点和固支节点不同
  f_points = np.random.choice(f_candidates, np.random.randint(2,4), replace=False) # 随机选择力的节点
  
  for point in f_points:
    if point < ppl:
      forces[dim*point//ppl, 0, np.random.randint(0,2)] = 1
    elif point < ppl*2:
      forces[dim, dim*(point%ppl)//ppl, np.random.randint(0,2)] = 1
    elif point < ppl*3:
      forces[dim*(point%ppl)//ppl, dim, np.random.randint(0,2)] = 1
    else:
      forces[0, dim*(point%ppl)//ppl, np.random.randint(0,2)] = 1
  bc_cond, force_cond = np.moveaxis(np.moveaxis(normals, -1, 0), 1, 2), \
              np.moveaxis(np.moveaxis(forces, -1, 0), 1, 2)
  density = np.random.choice(np.arange(0.3,0.7,0.1))
  return Problem(normals, forces, density), bc_cond, force_cond

def generate_dataset(dim, numbers, factor=12, opt_steps=20):
  '''生成数据集'''
  dens_full, disp_full, sens_full, bc_conds, f_conds= [], [], [], [], []
  i = 0
  while i < numbers:
    problem, bc_cond, f_cond = generate_square_problem(dim, factor)
    frames, sens, u = run_topo_simp_decompose(args=specified_task(problem, opt_steps=opt_steps))
    if np.any(np.array(sens)>0): # 如果灵敏度大于0，说明生成的结构异常，重新生成
      continue
    bc_conds.extend(
        np.broadcast_to(bc_cond, (len(frames), 2, dim+1, dim+1)))
    f_conds.extend(
        np.broadcast_to(f_cond, (len(frames), 2, dim+1, dim+1)))
    disp_full.extend(u)
    sens_full.extend(sens)
    dens_full.extend(frames)
    i += 1
    print(i,'structure generated')
  # 使用xarray库来打包数据 https://docs.xarray.dev/en/stable/index.html
  ds = xr.Dataset({
        'dens_full': (('number', 'nely', 'nelx'), dens_full),
        'disp_full': (('number', '2*nnpy*nnpx'), disp_full),
        'sens_full': (('number', 'nely', 'nelx'), sens_full),
        'bc_cond': (('number', 'channel', 'nnpy', 'nnpx'), bc_conds),
        'f_cond': (('number', 'channel', 'nnpy', 'nnpx'), f_conds),
    }, coords={'number': np.arange(len(dens_full))})
  return ds

In [7]:
def generate_reduced_dataset(densities, bc_conds, f_conds, factor):
  '''生成维度缩减后的结构及相应单元密度'''
  dens_red, disp_red, sens_red = [], [], []
  bc_conds, f_conds = np.array(bc_conds), np.array(f_conds)
  normals = np.moveaxis(np.moveaxis(bc_conds, 1, 3), 1, 2)
  forces = np.moveaxis(np.moveaxis(f_conds, 1, 3), 1, 2)
  for density, normal, force in zip(densities, normals, forces):
    problem = Problem(normal, force, 0.5)
    dens = average_downsample(density, factor=factor)
    reduced_problem = scale_reduction_square(problem, factor=factor)
    u, sens = fem_solver(dens, specified_task(reduced_problem))
    u = standardize(u)
    dens_red.append(dens)
    disp_red.append(u)
    sens_red.append(sens)
  ds = xr.Dataset({
        'dens_red': (('number', 'y_red', 'x_red'), dens_red),
        'disp_red': (('number', 'u_red'), disp_red),
        'sens_red': (('number', 'y_red', 'x_red'), sens_red),
    }, coords={'number': np.arange(len(dens_red))})
  
  return ds

由维度缩减后的数据集生成切块后的数据集：

In [9]:
def elements_to_block(images, block_size=2):
  ''' cut (n, nely, nelx) images  into 
  (n*(nely//block_size)*(nelx//blocksize), block_size, block_size)) blocks'''
  if len(images.shape) == 2:
    nely, nelx = images.shape[0], images.shape[1]
  else:
    nely, nelx = images.shape[1], images.shape[2]
  if nely % block_size != 0 or nelx % block_size != 0:
    raise ValueError(
    f'shape ({nely},{nelx}) cannot be reduced accurately by {factor}x')
  
  num_of_slice = nelx//block_size
  return np.moveaxis(images.reshape(-1,block_size,num_of_slice,block_size),1,2).reshape(-1,block_size,block_size)

def plot_blocks(columns, blocks, cmap='Greys'):    
    rows = blocks.shape[1]//columns + 1
    fig = plt.figure(figsize=(10, 10))
    for i in range(blocks.shape[0]):
      fig.add_subplot(rows, columns, i+1, frame_on=True)
      plt.imshow(blocks[i], cmap=cmap)
      plt.axis('on')

def plot_blocks(nelx, blocks, cmap='Greys'):    
    columns = nelx//blocks.shape[-1]
    rows = blocks.shape[0]//columns
    fig, axes = plt.subplots(rows, columns, figsize=(10,10))
    i = 0
    for ax in axes.flat:
      im = ax.imshow(blocks[i], vmin=0, vmax=15, cmap=cmap)
      i += 1
    fig.colorbar(im, ax=axes.ravel().tolist())
    plt.show()


def full_disp_to_block(image, block_size=2):
  '''rolling window with shape (block_size+1,block_size+1) and stride (block_size,block_size)'''
  return np.array(view_as_windows(image, window_shape=(block_size+1,block_size+1), step=(block_size,block_size))).reshape(-1,block_size+1,block_size+1)
def coarse_disp_to_block(image):
  '''rolling window with shape (2,2) and stride (1,1)'''
  return np.array(view_as_windows(image, window_shape=(2,2), step=(1,1))).reshape(-1,2,2)

def generate_block_dataset(ds_full, ds_red, block_size):
  dens_full = ds_full.dens_full.values # shape (number, nely, nelx)
  sens_full = ds_full.sens_full.values
  disp_full = ds_full.disp_full.values # shape (number, nnpy*nnpx*2)
  dens_red = ds_red.dens_red.values # shape (number, nely//block_size, nelx//block_size)
  sens_red = ds_red.sens_red.values
  disp_red = ds_red.disp_red.values

  dens_block = elements_to_block(dens_full, block_size=block_size)
  sens_block = elements_to_block(sens_full, block_size=block_size)
  dens_red_block = dens_red.ravel()
  sens_red_block = sens_red.ravel()

  ux_full_block, uy_full_block, ux_red_block, uy_red_block = [], [], [], []
  nely, nelx = dens_full.shape[1], dens_full.shape[2]
  for u in disp_full:
    ux, uy = separate_u(u, nelx, nely)
    ux_block = full_disp_to_block(ux, block_size=block_size)
    uy_block = full_disp_to_block(uy, block_size=block_size)
    ux_full_block.append(ux_block)
    uy_full_block.append(uy_block)

  for u in disp_red:
    ux, uy = separate_u(u, nelx//block_size, nely//block_size)
    ux_block = coarse_disp_to_block(ux)
    uy_block = coarse_disp_to_block(uy)
    ux_red_block.append(ux_block)
    uy_red_block.append(uy_block)
  
  ux_full_block = np.array(ux_full_block).reshape(-1,block_size+1, block_size+1)
  uy_full_block = np.array(uy_full_block).reshape(-1,block_size+1, block_size+1)
  ux_red_block = np.array(ux_red_block).reshape(-1,2,2)
  uy_red_block = np.array(uy_red_block).reshape(-1,2,2)

  ds = xr.Dataset({
        'dens_block': (('number', 'yblock', 'xblock'), dens_block),
        'sens_block': (('number', 'yblock', 'xblock'), sens_block),
        'dens_red_block': (('number',), dens_red_block),
        'sens_red_block': (('number',), sens_red_block),
        'ux_full_block': (('number', 'yblock+1','xblock+1'), ux_full_block),
        'uy_full_block': (('number', 'yblock+1','xblock+1'), uy_full_block),
        'ux_red_block': (('number', 'yfixed2','xfixed2'), ux_red_block),
        'uy_red_block': (('number', 'yfixed2','xfixed2'), uy_red_block)
    }, coords={'number': np.arange(len(dens_block))})
 
  return ds
  

------------------------------------------------------------------------------

由节点位移和单元密度反算灵敏度，验证有限元计算、分块方式是否正确：

In [12]:
def disp_to_sens(dens, uxs, uys, penal=3):
  ''' input density elements map(nelx, nely), its corresponding ux&uy map(nnpx,nnpy)'''
  ke = get_stiffness_matrix(1,0.3)
  sens = []
  for den, ux, uy in zip(dens.ravel(), view_as_windows(uxs, window_shape=(2,2), step=(1,1)).reshape(-1,2,2), view_as_windows(uys, window_shape=(2,2), step=(1,1)).reshape(-1,2,2)):
    ux, uy = ux.squeeze(), uy.squeeze()
    u = np.array([ux[0][0], uy[0][0], ux[0][1], uy[0][1], ux[1][1], uy[1][1], ux[1][0], uy[1][0]])
    sen = -penal*den**(penal-1) * np.matmul(np.matmul(u.T, ke), u)
    sens.append(sen)
  return np.array(sens).reshape(dens.shape)

array(-0.00838057)